In [2]:
# header for 2018-1 kernel
from pyCHX.chx_packages import *
%matplotlib notebook
plt.rcParams.update({'figure.max_open_warning': 0})
plt.rcParams.update({ 'image.origin': 'lower'   })
plt.rcParams.update({ 'image.interpolation': 'none'   })
import pickle as cpk
#from pyCHX.chx_xpcs_xsvs_jupyter_V1 import *
#%run /home/yuzhang/pyCHX_link/pyCHX/chx_generic_functions.py
#%matplotlib inline
import papermill as pm

In [3]:
#from pyCHX.XPCS_SAXS import get_QrQw_From_RoiMask

In [4]:
# import database -> should be hidden from user in same package....
import datetime
import pymongo 
from tqdm import tqdm
from bson import ObjectId
import matplotlib.patches as mpatches
from IPython.display import clear_output
cli = pymongo.MongoClient('xf11id-ca')    
samples_2 = cli.get_database('samples').get_collection('samples_2')
data_acquisition_collection = cli.get_database('samples').get_collection('data_acquisition_collection')
beamline_pos = cli.get_database('samples').get_collection('beamline_pos')
from databroker import Broker                                                   
db = Broker.named('temp')  # for real applications, 'temp' would be 'chx' 
print('available databases:')
print(cli.database_names())
print('\n available collection in database samples:')
print(cli.samples.collection_names())

available databases:
['amostra', 'chx-simulation-assetstore', 'chx-simulation-metadatastore', 'datastore', 'filestore', 'local', 'metadatastore', 'metadatastore-production-v1', 'samples']

 available collection in database samples:
['samples', 'data_acquisition_collection', 'samples_2', 'debug', 'beamline_pos']


/opt/conda_envs/analysis-2019-1.2-chx/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  from ipykernel import kernelapp as app
/opt/conda_envs/analysis-2019-1.2-chx/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.


In [5]:
def _chx_analysis_data( uid,  
        #template_pipeline = '/home/yuzhang/analysis/2019_1/commisionning/XPCS_Single_2019_V1_SQRange.ipynb', 
        template_pipeline = '/home/yuzhang/analysis/2019_1/petrash/XPCS_Single_2019_V2.ipynb',                
        outDir = '/home/yuzhang/analysis/2019_1/petrash/ResPipelines/',
                      ):
    ''' YG. Octo 6, 2018, Compress a eiger data using papermail
    Input:
        uid: string, the uique data id
        force compress: if True, will force to compress data no matter the data was compressed already
        The default compress pipeline  
        template_pipeline: str, the filename of the template pipeline
        outDir:str, the path for the output pipeline
    Output:
        save the current pipeline to outDir
    '''
    output_pipeline = outDir +   template_pipeline.split('/')[-1] + '_%s.ipynb'%uid 
    pm.execute_notebook(
        template_pipeline, output_pipeline,         
        parameters = dict( uid = uid,  ),
                    kernel_name= None, report_mode=True )      
    
    

In [6]:
if True:    
    temp1 = data_acquisition_collection.find_one({'_id':'general_list'})['compression_completed']
    temp2= data_acquisition_collection.find_one({'_id':'general_list'})['analysis_completed']
    #temp3= data_acquisition_collection.find_one({'_id':'general_list'})['analysis_failed']
    temp4= data_acquisition_collection.find_one({'_id':'general_list'})['analysis_failed_userX']
    s2 = set(temp2)
    #s3 = set(temp3)
    s4 = set(temp4)       
    #######################################
    #######Get uids to be compressed#######    
    uids = [x for x in temp1 if x not in s2 and x not in s4] 
    print(uids)

[]


In [ ]:
temp2[-1]


In [11]:
def get_masked_analysis_database( start_uid ):
    '''Give the uid for the first running and get the masked database'''
    temp1 = data_acquisition_collection.find_one({'_id':'general_list'})['compression_completed']
    for i,  t in enumerate(temp1):
        if t == start_uid:
            print(i,t)
            start_id = i    
    masked = data_acquisition_collection.update_one( 
           {'_id':'general_list'},{'$set':{'analysis_failed_userX':  temp1[:start_id] }   })
     

In [12]:
#get_masked_analysis_database( start_uid = 'b2dbbb62-633e-45af-ac33-d64bb3fdd373' )
#get_masked_analysis_database( start_uid ='40920a78-750f-4812-a604-601d2e6677dc')
#get_masked_analysis_database( start_uid ='70a31366-045e-40ad-8d18-281a9b8729a2')
#get_masked_analysis_database( start_uid ='e2d0ae81-ee8d-4b6c-9f32-aefc50b09b9a')

In [13]:
#data_acquisition_collection.update_one( {'_id':'general_list'},{'$set':{'analysis_failed_petrash':  temp1[:797] }   })

In [14]:
#temp1

In [15]:
#temp4.append(['2891d947-58b1-4db8-9b7e-c93f94259e78',   'f34aceea-dacb-439f-9e91-94bc37156354',          '0639c0c3-7257-436f-9bfe-b964830e3823']   )
#data_acquisition_collection.update_one({'_id':'general_list'},{'$set':{'analysis_failed_Surita':tem}})
#data_acquisition_collection.update_one({'_id':'general_list'},{'$set':{'analysis_failed':[]}})



# faking data analysis from compressed_uid list in data-acquisition database 

In [16]:
#data_acquisition_collection.update_one( {'_id':'general_list'}, {'$set':{'analysis_in_progress': [] }}) 
#data_acquisition_collection.find_one({'_id':'general_list'})['analysis_in_progress']


In [17]:
uids

[]

In [ ]:
# get list of uids
end_of_compression_key='none' # stops if only this key is left in compressed_uid_list, 'none': not looking for key, just for empty list timeout
empty_list_timeout= 3600 * 24 * 7 #[s] stops if compressed_uid_list is empty for x s

time_count=0
run_condition = True
while run_condition:
    clear_output()
    temp1 = data_acquisition_collection.find_one({'_id':'general_list'})['compression_completed']
    temp2= data_acquisition_collection.find_one({'_id':'general_list'})['analysis_completed']
    temp3= data_acquisition_collection.find_one({'_id':'general_list'})['analysis_failed_userX']
    temp4 = data_acquisition_collection.find_one({'_id':'general_list'})['analysis_in_progress']
    #data_acquisition_collection.find_one({'_id':'general_list'})['analysis_failed']
    s2 = set(temp2)
    s3 = set(temp3)
    s4 = set( temp4 )   
    #######################################
    #######Get uids to be compressed#######    
    uids = [x for x in temp1 if x not in s2 and x not in s3] 
    ######################################
    N = len(uids)
    if N>=3: # list of uids is not empty
        print('uid list for analysis is NOT empty, found '+str(len(uids))+' uids awaiting analysis.')
        time_count=0
        if end_of_compression_key != 'none' and uids[-1] == end_of_compression_key: #looking for a stop key, next uid up IS the stop key
            run_condition = False
            print('Stop Key for analysis detected!')
            
        else:
            print('Doing data analysis for uid '+uids[-1])
            
            #######################################
            #for ics in tqdm(range(100)):
            #    time.sleep(.35)
            ########################################
            uid = uids[-1]
            if uid not in s4:
                try:                    
                    temp4.append( uid )
                    data_acquisition_collection.update_one({'_id':'general_list'},{'$set':{ 'analysis_in_progress': temp4}})
                     
                    _chx_analysis_data( uid )                 
                        # update list of compressed uids:
                    temp2= data_acquisition_collection.find_one({'_id':'general_list'})['analysis_completed']     
                    temp2.append(   uid  )
                    data_acquisition_collection.update_one({'_id':'general_list'},{'$set':{ 'analysis_completed': temp2}})
                   
                except:
                    temp3= data_acquisition_collection.find_one({'_id':'general_list'})['analysis_failed_userX']
                    temp3.append(   uid )
                    data_acquisition_collection.update_one({'_id':'general_list'},{'$set':{ 'analysis_failed_userX': temp3}})
                
                ##remove this uid from  uid in analysis_in_progress
                temp4 = data_acquisition_collection.find_one({'_id':'general_list'})['analysis_in_progress']
                tx = [u for u in temp4  if u!=uid ]
                data_acquisition_collection.update_one( {'_id':'general_list'},
                                                           {'$set':{'analysis_in_progress':  tx }   }) 

                
    else:
        if time_count > empty_list_timeout:
            print('uid list for analysis was empty for > '+str(empty_list_timeout)+'s -> stop looking for new uids')
            run_condition = False
        else:
            time_count=time_count+5
            print('list of uids for analysis is emtpy...going to look again in 5s.')
            time.sleep(5)

list of uids for analysis is emtpy...going to look again in 5s.
